# Lab3: split-apply-combine in pandas & data visualization

Question:

Do Netflix subscribers prefer older or newer movies? Are there other factors that affect this preference such as *rating*?

Task:

- Split the dataset into groups, one for each year, and then to compute one or several summary statistics.
- See whether this statistic increases over the years.


Source: https://github.com/datacamp/community-groupby

## Data Exploration with pandas

### Import your data

In [ ]:
# Import packages and set visualization style
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
sns.set()
%matplotlib inline

In [ ]:
# Import data and check out head of DataFrame
df = pd.read_csv('netflix.csv')
df.head()

### Data Exploratory Analysis

Applying `pandas` DataFrame `.info()` below shows that you have 1000 rows and 7 columns of data, but also that the column of interest, `user_rating_score`, has only 605 non-null values. This means that there are 395 missing values:

In [ ]:
# Check out info of DataFrame
df.info()

### Dispersion Matrix plot

In [ ]:
# Drop rows with missing values and drop duplicate
df.dropna(inplace=True)
df.drop_duplicates(inplace=True)

# Visualize pairplot of df (similar to pairs() in R)
sns.pairplot(df, hue='rating');

In [ ]:
sns.pairplot?

In [ ]:
# Get summary stats of df
df.describe()

## Groupby (split-apply-combine) to answer the question

<a id='step_1'></a>

Let us first use a *groupby* method to split the data into groups, where each group is the set of movies released in a given year. 

In [ ]:
# Group by year
df_by_year = df.groupby('release_year')

This creates a *groupby* object:

In [ ]:
# Check type of GroupBy object
type(df_by_year)

Let us compute summary statistics of our response variable `user_rating_score` in order to capture the main characteristics for each value of `release_year`.

In [ ]:
(df.groupby('release_year')
    .describe())

We can depict the above information using a boxplot as follows:

In [ ]:
# Create boxplot  via factor plot
sns.factorplot(x="release_year", y="user_rating_score",data=df, kind="box")

Let's obtain a summary statistic, such as the mean and the compare agains the exmplanatory variables `release_year` and `rating`.

In [ ]:
(df.groupby(['release_year','rating'])
   .median())

In [ ]:
df_median = (df.groupby(['release_year','rating'])
               .apply(lambda x:x.user_rating_score.median())) # behaves similar to summarise()
df_median.head() 

In [ ]:
# To apply an arbitrary function the lambda function should return a new series
df1 = (df.groupby(['release_year','rating'])
               .apply(lambda x:pd.Series({'median_score':x.user_rating_score.median()})))
df1.head()

## `%>%`'ing in pandas 

- Method chaining means to call methods on an object one after another -- it is very popular in data science.

- Suppose we now want to compute an arbitrary function (e.g., the IQR) and the median along all possible values of `release_year` and `rating` for which the number of observations is positive.

In [ ]:
# To apply an arbitrary function the lambda function should return a new series
df2 = (df.groupby(['release_year','rating']) # combine step
               .apply(lambda x:pd.Series({'n':x.user_rating_score.count(),
                                          'median':x.user_rating_score.median(),
                                          'q1':x.user_rating_score.quantile(0.25),
                                          'q3':x.user_rating_score.quantile(0.75)}))
               .query('n>0') # similar to filter() in dplyr
               .assign(IQR_score=lambda x: x.q3-x.q1) # similar to mutate() in dplyr
      .sort_index()) 
df2

## From wide to long format

- We now want to create a line plot `user_rating_score` against `release_year`of the median along with inter-quartile range (i.e., plotting q3 and q1) but identifying each line according to `rating`. 
- To do so, it is more convenient to convert our previous DataFrame object from wide to long format.

In [ ]:
df3 = (pd.melt(df2.reset_index(), # notice that we have to add reset index
              col_level=0,
              id_vars=['release_year','rating'],
              value_vars=['median','q1','q3'],
              value_name='SummaryStatistic'))
df3.head()

In [ ]:
# Another less obscure option using stack()
df2.stack()

In [ ]:
df2.stack().reset_index()

In [ ]:
df3 = (df2.stack()
    .reset_index()
    .rename(index=str, columns={"level_2": "SummaryStatistic", 0: "value"}))
df3